import pandas as pd
from nltk import *

from framework import *
from ingredients import *
from helpers import *
from AutoCrawl import *
from steps import *

def getVerbs(text):
    methodlist = set()
    #sentences = text.split("\n")
    for sentences in text:
        method = sentence_verbs(sentences)
        methodlist = methodlist.union(set(method))
    return list(methodlist)

def sentence_verbs(sentence):
    vblist = []
    subsentencelist = re.split('[;.:]', sentence)
    for subs in subsentencelist:
        vb = subsentence_verbs(subs)
        vblist = vblist + vb
    return vblist
        
def subsentence_verbs(subsentence):
    tokens = word_tokenize(subsentence)
    tagtup = nltk.pos_tag(tokens)
    methodlist = []
    for i in range(len(tagtup)):
        if i == 0 and tagtup[0][1] not in ["RB", "DT"]:
            methodlist.append(tagtup[i][0].lower())
        elif tagtup[i][1] in ["VB", "VBZ"] and tagtup[i][0] not in ['is', 'are', 'am', 'a', 'an']:
            methodlist.append(tagtup[i][0].lower())
    return methodlist

def getLOfMethods(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [x.lower() for x in content]
    return content

def isCookingMethod(verblist, allmethods):
    methods = []
    for verb in verblist:
        if verb in allmethods:
            methods.append(verb)
    return methods

def isPrepMethod(verblist, cookingmethod):
    prepmethod = list(set(verblist) - set(cookingmethod))
    return prepmethod

def mainIngCookingMethod(mainingr, sentences):
    cookingList = set()
    for sentence in sentences:
        if mainingr in sentence.lower():
            method = sentence_verbs(sentence)
            cooking = isCookingMethod(method, allmethods)
            cookingList = cookingList.union(cooking)

    return list(cookingList)

def parse_max_cook_time(get_times_result):
    timevalue = 0
    if isinstance(get_times_result, str) == True:
        splitted = get_times_result.split(" ")
        if 'hour' in get_times_result:
            timevalue += 3600 * int(splitted[len(splitted)-2])
        elif 'minute' in get_times_result:
            timevalue += 60 * int(splitted[len(splitted)-2])
        elif 'second' in get_times_result:
            timevalue += int(splitted[len(splitted)-2])
        else:
            timevalue += 0
    return timevalue

def longestCookingMethod(sentences):
    minimumtime = 0
    for sentence in sentences:
        cookingtimestring = get_times(sentence)
        cookingtimestep = parse_max_cook_time(cookingtimestring)
        if cookingtimestep > 0 and cookingtimestep > minimumtime:
            maincookingverb = sentence_verbs(sentence)
            maincookingmethod = isCookingMethod(maincookingverb, allmethods)
            if len(maincookingmethod) > 0:
                minimumtime = cookingtimestep
    return maincookingmethod
    
def getDirections(text):
    recipeText = ""
    recipeText = text[text.index('Directions:\n') + 1:len(text)]
    return recipeText
    
def getMethodsDict(text, methodlist):
    recipeDirectons = getDirections(text)
    pri_method = longestCookingMethod(recipeDirectons)
    all_methods = getVerbs(recipeDirectons)
    addi_method = isPrepMethod(methodlist, pri_method)
    methods_dict = {}
    methods_dict['primary'] = pri_method
    methods_dict['preparatory'] = addi_method
    return methods_dict

mainingr = 'cauliflower'
def ingredientMentionCount(mainingr, sentences):
    mentioned = 0
    countDict = dict()
    for sentence in sentences:
        if mainingr in sentence.lower():
            mentioned += 1
    countDict[mainingr] = mentioned 
    return countDict

ingredientMentionCount(mainingr, sentences)

In [1]:
from framework import *

In [7]:
#recipeText = GetData("https://www.allrecipes.com/recipe/221227/honey-brined-fried-chicken-breasts/")
recipeText = GetData("https://www.allrecipes.com/recipe/219981/crab-stuffed-lobster-tail/")
#recipeText = GetData("https://www.allrecipes.com/recipe/8758/white-cheese-chicken-lasagna/")
#recipeText = GetData("https://www.allrecipes.com/recipe/256875/cheesy-sausage-lasagna-soup/")

Recipe File already exists


list_methods = getLOfMethods('ListOfMethods.txt')
allmethods = [item.lower() for item in list_methods]
recipeText = GetData("https://www.allrecipes.com/recipe/219981/crab-stuffed-lobster-tail/")



getMethodsDict(recipeText, allmethods)

In [8]:
GetIngredients(recipeText)
#recipeText

[{'name': 'lobster tails',
  'quantity': '2',
  'measurement': 'NoItem',
  'Preparation': 'split along the center top',
  'essential': 'lobster'},
 {'name': 'butter',
  'quantity': '2',
  'measurement': 'teaspoon',
  'Preparation': 'melted',
  'essential': None},
 {'name': 'buttery round crackers',
  'quantity': '15',
  'measurement': 'NoItem',
  'Preparation': 'crushed',
  'essential': None},
 {'name': 'jumbo lump crabmeat',
  'quantity': '1/2',
  'measurement': 'cup',
  'Preparation': 'NoItem',
  'essential': 'crabmeat'},
 {'name': 'clarified butter',
  'quantity': '1/4',
  'measurement': 'cup',
  'Preparation': 'clarified ',
  'essential': 'butter'},
 {'name': 'chopped fresh parsley leaves',
  'quantity': '1',
  'measurement': 'tablespoon',
  'Preparation': 'chopped ',
  'essential': None},
 {'name': 'seafood seasoning such',
  'quantity': '',
  'measurement': 'teaspoon',
  'Preparation': 'NoItem',
  'essential': None},
 {'name': 'garlic',
  'quantity': '1',
  'measurement': 'clove'

In [9]:
GetSteps(recipeText)

[[{'methods': ['preheat'],
   'tools': ['oven'],
   'ingredients': [],
   'times': None,
   'conditions': None,
   'raw': 'Preheat oven to 425 degrees F (220 degrees C).'}],
 [{'methods': ['lift', 'pull', 'rest'],
   'tools': [],
   'ingredients': ['lobster'],
   'times': None,
   'conditions': None,
   'raw': 'Pull the edges of the split lobster shells apart and gently lift the tail meat to rest above the shells.'},
  {'methods': ['place'],
   'tools': [],
   'ingredients': ['lobster tails'],
   'times': None,
   'conditions': None,
   'raw': 'Place the prepared lobster tails on a baking sheet.'}],
 [{'methods': ['brush'],
   'tools': ['teaspoon', 'spoon'],
   'ingredients': ['butter'],
   'times': None,
   'conditions': None,
   'raw': 'Brush each portion of tail meat with 1 teaspoon melted butter.'}],
 [{'methods': ['mix'],
   'tools': ['bowl', 'cup'],
   'ingredients': ['crackers',
    'seafood seasoning',
    'lemon zest',
    'lemon juice',
    'butter',
    'garlic',
    'salt',

In [10]:
GetMethods(recipeText)

{'primary': ['bake'],
 'other': ['shape',
  'rest',
  '``',
  'read',
  'preheat',
  'doesnt',
  'place',
  'mix',
  'fall',
  'lift',
  'press',
  'brush',
  'pull']}

In [11]:
Transformation(recipeText,"veg")

[[{'name': 'tempeh',
   'quantity': '2',
   'measurement': 'NoItem',
   'Preparation': 'split along the center top',
   'essential': 'lobster'},
  {'name': 'butter',
   'quantity': '2',
   'measurement': 'teaspoon',
   'Preparation': 'melted',
   'essential': None},
  {'name': 'buttery round crackers',
   'quantity': '15',
   'measurement': 'NoItem',
   'Preparation': 'crushed',
   'essential': None},
  {'name': 'fake crabmeat',
   'quantity': '1/2',
   'measurement': 'cup',
   'Preparation': 'NoItem',
   'essential': 'crabmeat'},
  {'name': 'clarified butter',
   'quantity': '1/4',
   'measurement': 'cup',
   'Preparation': 'clarified ',
   'essential': 'butter'},
  {'name': 'chopped fresh parsley leaves',
   'quantity': '1',
   'measurement': 'tablespoon',
   'Preparation': 'chopped ',
   'essential': None},
  {'name': 'seafood seasoning such',
   'quantity': '',
   'measurement': 'teaspoon',
   'Preparation': 'NoItem',
   'essential': None},
  {'name': 'garlic',
   'quantity': '1',


#txtfilename = 'methods.csv'
txtfilename = 'proteins.csv'
directory = "./dictionary/" + txtfilename
masterdata = loadTransformTable(directory)

list_methods = getLOfMethods('./dictionary/methods.csv')
methodlist = [item.lower() for item in list_methods]
recipeDirectons = getDirections(recipeText)
pri_method = longestCookingMethod(recipeDirectons, methodlist)
all_methods = getVerbs(recipeDirectons)
addi_method = isPrepMethod(methodlist, pri_method)
methods_dict = {}
methods_dict['primary'] = pri_method
methods_dict['other'] = addi_method

tokens = nltk.word_tokenize('Add chicken breasts, and weigh them down with a plate to keep them completely submerged.'.lower())
essential_ingr = [get_essentials(ingredient['name']) for ingredient in ingredients if get_essentials(ingredient['name']) in tokens]

substep =   {'raw': 'Add chicken breasts, and weigh them down with a plate to keep them completely submerged.',
   'methods': ['keep', 'weigh', 'add'],
   'ingredients': [],
   'tools': [],
   'times': None,
   'conditions': None}
substep['ingredients'] = substep['ingredients'] + essential_ingr
substep


In [15]:
from helpers import *
masterdata = loadTransformTable("./dictionary/proteins.csv")
protype = masterdata[masterdata['protein'] == 'chicken']['vegan_dim'].item()
protype

'animal'

In [16]:
ingredients = GetIngredients(recipeText)
ingredients

[{'name': 'lasagna noodles',
  'quantity': '9',
  'measurement': 'NoItem',
  'Preparation': 'NoItem',
  'essential': None},
 {'name': 'butter',
  'quantity': '1/2',
  'measurement': 'cup',
  'Preparation': 'NoItem',
  'essential': 'butter'},
 {'name': 'onion,',
  'quantity': '1',
  'measurement': 'NoItem',
  'Preparation': 'chopped',
  'essential': None},
 {'name': 'garlic,',
  'quantity': '1',
  'measurement': 'clove',
  'Preparation': 'minced',
  'essential': None},
 {'name': 'all-purpose flour',
  'quantity': '1/2',
  'measurement': 'cup',
  'Preparation': 'NoItem',
  'essential': None},
 {'name': 'salt',
  'quantity': '1',
  'measurement': 'teaspoon',
  'Preparation': 'NoItem',
  'essential': None},
 {'name': 'chicken broth',
  'quantity': '2',
  'measurement': 'cup',
  'Preparation': 'NoItem',
  'essential': 'chicken'},
 {'name': 'milk',
  'quantity': '1 1/2',
  'measurement': 'cup',
  'Preparation': 'NoItem',
  'essential': 'milk'},
 {'name': 'shredded mozzarella cheese,',
  'qua

In [17]:

replacementdict = ingredToVeg(ingredients, masterdata)
replacementdict

{'chicken broth': 'tempeh', 'chicken': 'tempeh'}

In [ ]:
steps = GetSteps(recipeText)

In [ ]:
replaceIngrInSteps(steps, replacementdict)

#ingfull = 'boneless skinless chicken breast halves'
ingfull2 = 'yummy chicken stock'
ingtokens = word_tokenize(ingfull2)
def bigramsMatchKeyword(ingtokens, keyword):
    keywordposition = ingtokens.index(keyword)
    ingbigrams = list(bigrams(ingtokens))
    gen_bigrams = []
    if keywordposition != 0:
        bg_before = ingbigrams[keywordposition-1][0] + " " + ingbigrams[keywordposition-1][1]
        gen_bigrams.append(bg_before)
    if keywordposition != len(ingtokens) - 1:
        bg_after = ingbigrams[keywordposition][0] + " " + ingbigrams[keywordposition][1]
        gen_bigrams.append(bg_after)
    return gen_bigrams

k = bigramsMatchKeyword(ingtokens, 'chicken')
k[1]+"s"

In [ ]:
list(ingbigrams)
#ingtokens.index('chicken')

In [ ]:
'stock' in 'chicken broth' or 'broth' in 'chicken broth'

In [ ]:
word_tokenize('lobster tails,')